In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.spatial import distance
import matplotlib.pyplot as plt
import seaborn as sns

import os.path as op
import os
import glob
import shutil
import datetime
import math

from nilearn import plotting
from nilearn import image
from nilearn import masking
import nilearn

In [ ]:
# Get [fear, disgust] maskfiles
proj_dir = 'PATH_TO_DIRECTORY'
path_to_masks = op.join(proj_dir, 'TIER/analysis_data/ROI_MASKS/')
maskfiles = []
# I am adding this next line for brevity in testing mode 
#maskfiles += glob.glob(path_to_masks + 'disgust*IFG.nii.gz')

# THE NEXT THREE LINES ARE REAL 
maskfiles += glob.glob(path_to_masks + 'disgust*')
maskfiles += glob.glob(path_to_masks + 'fear*')
maskfiles += glob.glob(path_to_masks + 'emo_reg*')

print(maskfiles)

In [ ]:
# Set up variables for main extraction
tier_dir = op.join(proj_dir, 'TIER')
copes_dir = op.join(tier_dir,'original_data','copes')
varcopes_dir = op.join(tier_dir,'original_data','varcopes')
zstats_dir = op.join(tier_dir,'original_data','zstats')
path_to_goodvoxel_masks = op.join(proj_dir, 'TIER/analysis_data/ROI_masks_goodvoxels/')

# create some directories for outputs 
top_voxel_mask_outputdir = op.join(proj_dir, 'output','subject_top_voxel_masks')
violinplots_dir = op.join(proj_dir, 'output', 'violinplot')
os.makedirs(top_voxel_mask_outputdir, exist_ok = True)


# set the contrasts 
cons = ['con_1_tgn-gt-cgn', 
        'con_2_cgd-gt-cgn',
        'con_3_cgf-gt-cgn',
        'con_4_cgf-gt-cgd',
        'con_5_cgd-gt-cgf',
        'con_6_cgn',
        'con_7_cgd',
        'con_8_cgf',
        'con_9_tgn', 
        'con_10_tgn-gt-cgd', 
        'con_11_tgn-gt-cgf']


# set the list of tasks 
tasks = ['read']
# there's only one for now, so we can skip a loop by setting it here. If you add more tasks, you need to build another layer of for-loops over each task
task = tasks[0]


ROIs = maskfiles
# just get the name of the masks (for plotting later)
masklabels = list(map(lambda x: x.split('/')[-1].split('.')[0], maskfiles))

# either take the whole ROI or just the top 100 voxels -- we'll do both and loop through each option
#num_top_voxels_options = ['whole', 100]
num_top_voxels_options = [100]

# Maximum number of folds (including the fold without any exclusions)
# Should = number of runs + 1
num_folds = 5

# get the info for subjects and exclusions 
master_data = op.join(proj_dir, 'data/subject_lists/EMOfd_subject_info_211026.csv')
df = pd.read_csv(master_data)

# this will be a variable in the output CSV .. 
sub_proj = 'MIT'

In [ ]:
filesubs = pd.unique(df.loc[(df.exclude_from_analysis == False), 'acquisitionID']).tolist()

In [ ]:
for roi in ROIs: 
    mask_img_fname = roi
    roi = op.basename(roi).split('.')[0]
    print('WORKING ON ROI: ', roi)
    # load the ROI image
    mask_img = image.load_img(mask_img_fname)
    mask_img_data = mask_img.get_fdata()
    
    for sub in filesubs:
        print('WORKING ON SUBJECT: ', sub)
        # initialize the subject-level mask where we'll combine across contrasts 
        # the goal being to find where varcope is zero across all contrasts, then exclude those from the ROI mask
        bad_voxel_mask = np.zeros_like(mask_img_data)
        
        for con in cons: 
            print('WORKING ON CONTRAST: ', con)
            
            # get the file name of the exclude-none varcope for this subject, for this contrast
            varcope_current = '{}/{}_{}_{}_exclude_none_{}_varcope.nii.gz'.format(varcopes_dir, sub, task, 'fold*', con)
            matches_varcope = glob.glob(varcope_current)

            if len(matches_varcope) > 1:
                print('There are duplicate fear files present.')
                pass
            elif len(matches_varcope) < 1:
                print(varcope_current, ' is missing!')
                pass
            else:
                # load the data for the varcope image 
                varcope_img = image.load_img(matches_varcope[0])
                varcope_data = varcope_img.get_fdata()
                
                #find where it is zero 
                varcopes_bool = np.abs(varcope_data) == 0.0
                # turn boolean result into 1/0 
                varcopes_zeros = varcopes_bool.astype(int)
                # add this to the subject-level mask 
                bad_voxel_mask += varcopes_zeros

        # now we want to find where the varcope was zero for ALL contrasts 
        bad_voxels_bool = bad_voxel_mask == len(cons)
        
        # invert to get the GOOD voxels 
        good_voxels_bool = np.invert(bad_voxels_bool)
        
        # turn that into a 1/0 mask 
        good_voxels = good_voxels_bool.astype(int)
        
        # make this into an image 
        good_voxel_img = image.new_img_like(mask_img, good_voxels)
        
        # now we need to mask the ROI image with this good-voxel image, to remove any ROI voxels that are not good: 
        good_ROI = image.math_img("img1 * img2", img1 = mask_img, img2 = good_voxel_img)
        
        # save this per-subject, per-roi mask of "good voxels in the ROI"
        good_ROI_out_fname = '{}/ROI_GOODVOX_{}_task-{}_{}.nii.gz'.format(path_to_goodvoxel_masks, sub, task, roi)

        good_ROI.to_filename(good_ROI_out_fname)
        print('WROTE FILE')
        print(good_ROI_out_fname)
        print('    ')





In [ ]:
print("Done!")